In [1]:
import os
import sys

parent_dir = os.path.abspath(os.path.join(os.pardir))
sys.path.append(parent_dir)

from libsql_graph_db import database as db
from dotenv import load_dotenv # type: ignore

load_dotenv()
db_url = os.getenv("LIBSQL_URL")
auth_token = os.getenv("LIBSQL_AUTH_TOKEN")


In [2]:
# To initialze the database and establish a connection with tursodb
db.initialize(db_url, auth_token)

Traceback (most recent call last):
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88  File "/home/anubhuti/Desktop/simple-graph-db/libsql_graph_db/database.py", line 79, in initialize
:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
    return atomic(_init, db_url, auth_token)  # type: ignore
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


In [3]:
# To add a single node
def insert_single_node(db_url, auth_token, new_label, new_node, new_node_id):
    try:
        db.atomic(db.add_node(new_label, new_node, new_node_id), db_url, auth_token)
    except Exception as e:
        assert False
new_node = {'subject': 'MES', 'type': ['person', 'Dr']}
new_label = "Mechanical Engineer"
new_node_id = 2

insert_single_node(db_url, auth_token, new_label, new_node, new_node_id)
    

In [4]:
# To add multiple nodes
def bulk_insert_operations(db_url, auth_token, labels, nodes):
    ids = []
    bodies = []
    for id, body in nodes.items():
        ids.append(id)
        bodies.append(body)

    # bulk add and confirm
    db.atomic(db.add_nodes(bodies, labels, ids), db_url, auth_token)

new_nodes = {
    2 : {'name': 'Peri', 'age': '90'},
    3 : {'name': 'Pema', 'age': '66'}
}
new_labels = ["Cancer Patient", "Tuberculosis Patient"]

bulk_insert_operations(db_url, auth_token, new_labels, new_nodes)

Traceback (most recent call last):
  File "/tmp/ipykernel_181443/2910530372.py", line 10, in bulk_insert_operations
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


In [6]:
# To find for a node
def find_a_node(db_url, auth_token, node):
            return db.atomic(db.find_node(node), db_url, auth_token)


find_a_node(db_url, auth_token, 2)

Traceback (most recent call last):
  File "/tmp/ipykernel_181443/3171091352.py", line 3, in find_a_node
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


{'subject': 'MES', 'type': ['person', 'Dr'], 'id': 2}

In [7]:
# For bulk upsert: Update and insert
def bulk_upsert(db_url, auth_token, labels, nodes):
    ids = []
    bodies = []
    for id, body in nodes.items():
        ids.append(id)
        bodies.append(body)

    db.atomic(db.upsert_nodes(bodies, labels, ids), db_url, auth_token)
nodes = {
    1 : {'name': 'Stanley', 'age': '30'},
    
    2 : {'name': 'Sheeran', 'type': ['singer', 'rich']}
}
labels = ["Lunch Box", "Pop Singer"]

bulk_upsert(db_url, auth_token, labels, nodes)
    

In [8]:
# For single upsert: Update and insert a node
def single_upsert(db_url, auth_token, label, body, id):
    
    db.atomic(db.upsert_node(id, label, body), db_url, auth_token)

body = {'name': 'Sheeran', 'type': ['singer', 'rich']}
label = "related as an"
id = 2
single_upsert(db_url, auth_token, label, body, id)

In [9]:
# Bulk connect and confirm
def bulk_node_connect(db_url, auth_token, labels, edges):
    sources = []
    targets = []
    attributes = []
    for src, tgts in edges.items():
        for target in tgts:
            tgt, label = target
            sources.append(src)
            targets.append(tgt)
            if label:
                attributes.append(label)
            else:
                attributes.append({})
        

    db.atomic(db.connect_many_nodes(
        sources, targets, label, attributes), db_url, auth_token)
    
edges = {
    1: [
        (3 , {'wealth':'1000 Billion'}),
        (1 , {'balance': '1000 rupees'})
    ]
}
edges_labels = ["Has", "Has"]

bulk_node_connect(db_url, auth_token, edges_labels, edges)
    


In [11]:
# Single connect and confirm
def single_node_connect(db_url, auth_token, source, target, label, attribute):
    db.atomic(db.connect_nodes(
        source, target,  label, attribute), db_url, auth_token)
    
source = 3
target = 1
label = "related as an"
attribute = {"relation": "enemy"}

single_node_connect(db_url, auth_token, source, target, label, attribute)

In [12]:
# To remove nodes
def remove_bulk_nodes(db_url, auth_token, ids):
        db.atomic(db.remove_nodes(ids), db_url, auth_token)

ids = [1, 4]

remove_bulk_nodes(db_url, auth_token, ids)


In [13]:
# To remove a single node
def remove_single_node(db_url, auth_token, id):
    db.atomic(db.remove_node(id), db_url, auth_token)

remove_single_node(db_url, auth_token, 2)

In [14]:
# To traverse the nodes
def traverse_nodes(db_url, auth_token, src, tgt):
    return db.traverse(db_url, auth_token, src, tgt)
traverse_nodes(db_url, auth_token, 3, 2)

Traceback (most recent call last):
  File "/home/anubhuti/Desktop/simple-graph-db/libsql_graph_db/database.py", line 552, in traverse
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
    return atomic(_traverse, db_url, auth_token)  # type: ignore
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


['3']

In [15]:
# To generate a query clause and find nodes
kv_name_like = db._generate_clause('name', predicate='LIKE')
print(db.atomic(db.find_nodes([kv_name_like], ('Pe%',)), db_url, auth_token))


Traceback (most recent call last):
  File "/tmp/ipykernel_181443/2893161704.py", line 3, in <module>
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


[{'name': 'Pema', 'age': '66', 'id': 3}]


In [16]:
# Visualization
from libsql_graph_db import visualizers
raw_path = "temp_path" +  "sample1.dot"
visualizers.graphviz_visualize(db_url, auth_token, raw_path, ['3'])

Traceback (most recent call last):
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:  File "/home/anubhuti/Desktop/simple-graph-db/libsql_graph_db/visualizers.py", line 61, in graphviz_visualize
    for edge in db.atomic(connections(i), db_url, auth_token):  # type: ignore
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime
88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
Traceback (most recent call last):
  File "/home/anubhuti/Desktop/simple-graph-db/libsql_graph_db/visualizers.py", line 75, in graphviz_visualize
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
    node = db.atomic(db.find_n

Here (2, '3', '3', 'Has', '{"wealth":"1000 Billion"}', '2024-04-01 11:10:42', '2024-04-01 11:10:42')
Here (4, '3', '3', 'related as an', '{"relation":"enemy"}', '2024-04-01 11:10:43', '2024-04-01 11:10:43')


In [17]:
# Visualization with bodies

raw_path = "temp_path" +  "bodies.dot"
path_with_bodies = db.traverse(db_url, auth_token, 2, with_bodies=True)
visualizers.graphviz_visualize_bodies(raw_path, path_with_bodies)

Traceback (most recent call last):
  File "/home/anubhuti/Desktop/simple-graph-db/libsql_graph_db/database.py", line 552, in traverse
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
    return atomic(_traverse, db_url, auth_token)  # type: ignore
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


In [19]:
# vector search node
def vector_search_node(db_url, auth_token, data, k):
    try:
        db.atomic(db.vector_search_node(data, k), db_url, auth_token)
    except Exception as e:
        assert False
        
data = {'subject': 'MES', 'type': ['person', 'Dr']}
k = 1
vector_search_node(db_url, auth_token, data, k)
    

Traceback (most recent call last):
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:  File "/tmp/ipykernel_181443/1772204510.py", line 4, in vector_search_node
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime
88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime


In [20]:
# vector search edge
def vector_search_edge(db_url, auth_token, data, k):
    try:
        db.atomic(db.vector_search_edge(data, k), db_url, auth_token)
    except Exception as e:
        assert False
        
data = {'subject': 'MES', 'type': ['person', 'Dr']}
k = 1
vector_search_node(db_url, auth_token, data, k)
    

Traceback (most recent call last):
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88:  File "/tmp/ipykernel_181443/1772204510.py", line 4, in vector_search_node
9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime
